In [1]:
from pulp import *
from pulp import LpProblem, LpVariable, LpMinimize, LpInteger, lpSum, value, LpBinary,LpStatusOptimal
import pulp
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore", message="Overwriting previously set objective.")
import utility
import docplex.mp.model
import docplex
import docplex_explainer
import mymetrics

In [2]:
# Load Dataset
dataset = datasets.load_iris()
dataset_name = 'Iris'
df = pd.DataFrame(dataset.data, columns = dataset.feature_names)

In [3]:
# Scale
scaler = MinMaxScaler()
scaler.fit(dataset.data)
scaled_df = scaler.transform(dataset.data)

In [4]:
# Get scaled bounds
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)

0.0 1.0


In [5]:
# Check if binary targets
df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
df_scaled['target'] = targets

Original Targets:  [0 1] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True


In [6]:
# Train model
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.3,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

clf = svm.SVC(kernel='linear')

# Train the model using the training set
clf.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = clf.predict(X_test)
print("Accuracy Linear:", metrics.accuracy_score(y_test, y_pred))

Accuracy Linear: 1.0


In [7]:
# Finding patterns classified as positive/negative
positive_indexes,negative_indexes = utility.find_indexes(clf, X_test, threshold=0)
print(f"Positive patterns = {len(positive_indexes)},\nNegative patterns = {len(negative_indexes)}")

Positive patterns = 30,
Negative patterns = 15


In [8]:
# Make a dataframe with the test data. For comparing Onestep against Twostep.
test_df_names = list(df.columns)
if 'target' not in test_df_names:
    test_df_names.append('target')
test_dataset = []
for instance, test_class in zip(X_test, y_test.astype('int32')):
    test_dataset.append(np.append(instance, test_class))
test_dataset_df = pd.DataFrame(np.asarray(test_dataset), columns=test_df_names)

In [9]:
# Parameter p value
p = 0.5

In [15]:
#Variables for results []
coverage_twostep = []
pos_exp_twostep = []
neg_exp_twostep = []

coverage_onestep = []
pos_exp_onestep = []
neg_exp_onestep = []
#Generate Explanations for the patterns classified as negative
for idx in  negative_indexes:
    
    #Twostep
    exp_ = docplex_explainer.twostep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            p = p,
            positive = False)
    neg_exp_twostep.append(exp_)
    coverage_twostep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp_)))
    
    #Onestep
    exp = docplex_explainer.onestep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            positive = False)
    neg_exp_onestep.append(exp)
    coverage_onestep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp)))
for i in range(len(coverage_onestep)):
    print(f'\ni={i}')
    print(coverage_onestep[i], neg_exp_onestep[i],)
    print(coverage_twostep[i], neg_exp_twostep[i])
mean_A= sum(coverage_onestep)/len(coverage_onestep)
mean_B= sum(coverage_twostep)/len(coverage_twostep)
print(mean_A,mean_B)


i=0
15 [[0.0, 1.0], [0.352021508800505, 1.0], [0.0, 0.11864406779661013], [0.0, 0.125]]
11 [[0.0, 1.0], [0.5137114203688846, 1.0], [0.0, 0.18765790040017305], [0.0, 0.16512991270501043]]

i=1
11 [[0.0, 1.0], [0.2124466800786586, 1.0], [0.0, 0.1016949152542373], [0.0, 0.04166666666666667]]
11 [[0.0, 1.0], [0.3483521524177307, 1.0], [0.0, 0.15963698877735816], [0.0, 0.07547278131851537]]

i=2
2 [[0.0, 1.0], [0.12950193481419145, 1.0], [0.0, 0.0847457627118644], [0.0, 0.0]]
11 [[0.0, 1.0], [0.4058866866274387, 1.0], [0.0, 0.2030092077528999], [0.0, 0.06825955326061245]]

i=3
1 [[0.0, 1.0], [0.10318727300710376, 1.0], [0.0, 0.06779661016949151], [0.0, 0.0]]
11 [[0.0, 1.0], [0.3113150229980089, 1.0], [0.0, 0.15675070808804703], [0.0, 0.0515190885407841]]

i=4
8 [[0.0, 1.0], [0.2164474399218625, 1.0], [0.0, 0.06779661016949151], [0.0, 0.08333333333333333]]
11 [[0.0, 1.0], [0.28835262731973316, 1.0], [0.0, 0.0982571577363213], [0.0, 0.10144297020982136]]

i=5
9 [[0.0, 1.0], [0.18613201827157

In [17]:
# Instance to be explained
instance = X_test[negative_indexes[7:8]]
instance

array([[0.08333333, 0.58333333, 0.06779661, 0.08333333]])

In [27]:
scaler.inverse_transform(np.atleast_2d(instance))

array([[4.6, 3.4, 1.4, 0.3]])

In [29]:
clf.predict(instance)

array([0])

In [23]:
#Onestep
onestep_exp = docplex_explainer.onestep(
        classifier = clf,
        dual_coef = clf.dual_coef_,
        support_vectors = clf.support_vectors_,
        intercept = clf.intercept_,
        lower_bound = lower_bound,
        upper_bound = upper_bound,
        data = (instance),
        positive = False)

In [25]:
onestep_exp

[[0.0, 1.0],
 [0.2164474399218625, 1.0],
 [0.0, 0.06779661016949151],
 [0.0, 0.08333333333333333]]

In [31]:
#Onestep Lowerbound
scaler.inverse_transform(np.atleast_2d([0,
                                       0.2164474399218625,
                                       0,
                                       0]))

array([[4.3       , 2.51947386, 1.        , 0.1       ]])

In [33]:
#Onestep Upperbound
scaler.inverse_transform(np.atleast_2d([1.0,
                                       1.0,
                                       0.06779661016949151,
                                       0.08333333333333333]))

array([[7.9, 4.4, 1.4, 0.3]])

In [35]:
#Twostep
twostep_exp = docplex_explainer.twostep(
        classifier = clf,
        dual_coef = clf.dual_coef_,
        support_vectors = clf.support_vectors_,
        intercept = clf.intercept_,
        lower_bound = lower_bound,
        upper_bound = upper_bound,
        data = (instance),
        p = p,
        positive = False)

In [37]:
twostep_exp

[[0.0, 1.0],
 [0.35085262731973305, 1.0],
 [0.0, 0.12509446594277981],
 [0.0, 0.11677149354157429]]

In [39]:
#Twostep Lowerbound
scaler.inverse_transform(np.atleast_2d([0,
                                       0.35085262731973305,
                                       0,
                                       0]))

array([[4.3       , 2.84204631, 1.        , 0.1       ]])

In [41]:
#Twostep Powerbound
scaler.inverse_transform(np.atleast_2d([1,
                                       1,
                                       0.12509446594277981,
                                       0.11677149354157429]))

array([[7.9       , 4.4       , 1.73805735, 0.38025158]])

In [43]:
len(mymetrics.calculate_coverage(test_dataset_df, onestep_exp))

8

In [45]:
len(mymetrics.calculate_coverage(test_dataset_df, twostep_exp))

13